In [1]:
!pip install matplotlib

^C
ERROR: Operation cancelled by user


## Import the required libraries

In [5]:
import cv2 
import numpy as np 
import random 
import os 
import matplotlib.pyplot as plt 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Layer, Input
import tensorflow as tf
import uuid


### Creating the Image paths
It is time to create folders to store anchor, positive and negative images. First, the folder path's are stored in the corresponding variables

In [4]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

Now we create the directories using the given folder paths.

In [3]:
#os.makedirs(POS_PATH)
#os.makedirs(NEG_PATH)
#os.makedirs(ANC_PATH)

The negative images have been collected from the Labelled Faces in the Wild (LFW) dataset. This dataset consists of over 13000 labelled faces collected from the web. Now, it is time to shift these images from their respective directories to the Negative directory that has been created

In [6]:
for directory in os.listdir('lfw'):
    if directory == '.DS_Store':
        continue
    for file in os.listdir(os.path.join('lfw', directory)):
        ex_path = os.path.join('lfw', directory, file)
        new_path = os.path.join(NEG_PATH, file)
        os.replace(ex_path, new_path)

## Collecting Anchor and Positive Images
Now, since the negative images have already been collected from the LFW dataset, the anchor and positive images are collected manually using the cv2 library. The anchor images and positive images are stored in the anchor and positive directories respectively. 

In [4]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[450:900, 750:1200, :]

    if cv2.waitKey(1) & 0XFF == ord('a'):
        img_name = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(img_name, frame)

    if cv2.waitKey(1) & 0XFF == ord('p'):
        img_name = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(img_name, frame)        
    
    cv2.imshow('Image Collection', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break 
cap.release()
cv2.destroyAllWindows()

In [6]:
#plt.imshow(frame)

### Storing file paths into a variable

In order to now build the model, 300 images from the anchor, positive and negative directories are selected at random. These are stored in separate variables

In [11]:
anchor = tf.data.Dataset.list_files(ANC_PATH + '/*.jpg').take(300)
positive= tf.data.Dataset.list_files(POS_PATH + '/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH + '/*.jpg').take(300)

### Preprocessing function
The anchor, positive and negative variables currently only contain the file paths to the individual images. However, in order to visualise the images, a preprocess function is constructed which takes the file path and returns the image in a normalized tensor format suitable for model input.

The function performs several key steps:

1. **File Reading**: Uses `tf.io.read_file()` to read the image file from the given path as raw bytes
2. **Image Decoding**: Converts the byte data into a tensor using `tf.io.decode_jpeg()` 
3. **Resizing**: Standardizes all images to 100x100 pixels using `tf.image.resize()` to ensure consistent input dimensions
4. **Normalization**: Divides pixel values by 255 to scale them from the range [0,255] to [0,1], which helps with model training stability and convergence

In [7]:
def preprocess(file_path): 
    byte_img = tf.io.read_file(file_path) ## reads file  
    img = tf.io.decode_jpeg(byte_img)  ## decodes the bytes into tensor
    img = tf.image.resize(img, (100,100)) ##resizing the image into (100,100)
    img = img/255
    return img

In [9]:
img = preprocess('data/anchor/07a799fa-679f-11f0-8ad3-b2f4ed02152c.jpg')
#plt.imshow(img)
print(img.shape)

(100, 100, 3)


### Constructing the dataset
Now in order to create the actual dataset, a dataset containing both positive and negative examples is created. The dataset contains both positive pairs (same person) and negative pairs (different people). This is done by adding label 1 to each positive pair and label 0 for each negative pair. 

In [15]:
positives = tf.data.Dataset.zip(anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor))))
negatives = tf.data.Dataset.zip(anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor))))
data = positives.concatenate(negatives)

In [16]:
samples = data.as_numpy_iterator()
example = samples.next()

Another function is created which takes as input the input image, validation image and the corresponding label adn then returns all of them preprocessed (using the function defined earlier. This is because since the data object created only contains the file paths. 

### Partioning into Train and Test datasets

In [17]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

In [19]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size = 1024)

In [20]:
train_data = data.take(round(len(data) *.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [21]:
test_data = data.skip(round(len(data) * .7))
test_data = data.take(round(len(data) *.3))
test_data = test_data.batch(16) 
test_data = test_data.prefetch(8)

### Creating the CNN backbone

The embedding layer takes images as input and converts them into feature vectors (embeddings). The model learns to create similar embeddings for images of the same person and different embeddings for images of different people. This allows the network to effectively compare faces by measuring the L1 distance between their embeddings.

In [109]:
def make_embedding():
    inp = Input(shape = (100,100,3), name = 'input_image')
    c1 = Conv2D(filters = 64, kernel_size = (10,10), activation = 'relu')(inp)
    p1 = MaxPooling2D(64, (2,2), padding = 'same')(c1)
    c2 = Conv2D(filters = 128, kernel_size = (7,7), activation = 'relu')(p1)
    p2 = MaxPooling2D(64, (2,2), padding = 'same')(c2)
    c3 = Conv2D(filters = 128, kernel_size = (4,4), activation = 'relu')(p2)
    p3 = MaxPooling2D(64,(2,2), padding = 'same')(c3)
    c4 = Conv2D(filters = 256, kernel_size = (4,4),activation = 'relu')(p3) 
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation = 'sigmoid')(f1)
    return Model(inputs = inp, outputs = d1, name = 'embedding')

In [110]:
embedding = make_embedding()
embedding.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)        │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 91, 91, 64)     │        19,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 40, 40, 128)    │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 17, 17, 128)    │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 6, 6, 256)      │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │    37,752,832 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,960,448 (148.62 MB)

 Trainable params: 38,960,448 (148.62 MB)

 Non-trainable params: 0 (0.00 B)

In [111]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [112]:
def make_siamese_model():
    input_img = Input(shape = (100,100,3), name = 'input_image')
    validation_img = Input(shape = (100,100,3), name = 'validation_image')

    siamese_layer = L1Dist()
    siamese_layer._name = 'distances'
    distances = siamese_layer(embedding(input_img), embedding(validation_img))
    classifier = Dense(1, activation = 'sigmoid')(distances)

    return Model(inputs = [input_img, validation_img], outputs = classifier, name = 'Siamese-model')
    

In [115]:
siamese_model = make_siamese_model()
siamese_model.summary()

Model: "Siamese-model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ validation_image    │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 4096)      │ 38,960,448 │ input_image[0][0… │
│ (Functional)        │                   │            │ validation_image… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_dist_5 (L1Dist)  │ (None, 4096)      │          0 │ embedding[4][0],  │
│                     │                   │            │ embedding[5][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │      4,097 │ l1_dist_5[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)

### Defining the loss function and the optimizer

In [37]:
loss_function = tf.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(1e-4)

In [38]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer = optimizer, siamese_model = siamese_model)

### Creating custom training loop using tf.GradientTape()

In [39]:
@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        X = batch[:2]
        y = batch[2]

        yhat = siamese_model(X, training = True)
        loss = loss_function(y,yhat)

    grad = tape.gradient(loss, siamese_model.trainable_variables)
    optimizer.apply_gradients(zip(grad, siamese_model.trainable_variables))

    return loss

In [40]:
def train(data,epochs):
    for epoch in range(1, epochs + 1):
        print(("Epoch {}/{}").format(epoch, epochs))
        progbar = tf.keras.utils.Progbar(len(data))

        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)

        if epoch % 10 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

In [41]:
EPOCHS = 50

In [42]:
train(train_data, EPOCHS)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step
Epoch 2/50


2025-07-29 23:49:07.888771: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


27/27 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step
Epoch 3/50


2025-07-29 23:50:30.239363: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


27/27 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step
Epoch 4/50


...

2025-07-30 00:53:03.955399: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


27/27 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step
Epoch 50/50


2025-07-30 00:54:26.402864: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


27/27 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step


2025-07-30 00:55:46.903827: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Evaluating the Model

In order to see if our model performs well, the precision and recall of the model's predictions are tested. 

In [43]:
from tensorflow.keras.metrics import Precision, Recall

In [44]:
test_inputs, test_val, y_true = test_data.as_numpy_iterator().next()

In [45]:
y_true

array([0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0.],
      dtype=float32)

In [46]:
predictions = siamese_model.predict([test_inputs, test_val])
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 970ms/step


array([[7.6781151e-11],
       [1.0000000e+00],
       [3.6550118e-10],
       [9.9999809e-01],
       [1.2250108e-11],
       [1.0000000e+00],
       [2.3620935e-08],
       [9.9997634e-01],
       [1.0000000e+00],
       [1.7891752e-11],
       [1.0000000e+00],
       [5.9903812e-08],
       [9.9993932e-01],
       [9.9999750e-01],
       [5.4349214e-10],
       [5.0861200e-07]], dtype=float32)

In [49]:
y_hat = [1 if pred > 0.5 else 0 for pred in predictions]

In [50]:
recall = Recall()
recall.update_state(y_true, y_hat)
recall.result().numpy()

1.0

In [51]:
precision = Precision()
precision.update_state(y_true,y_hat)
precision.result().numpy()

1.0

Recall and Precision both being 1 is a good sign even though these results are only for a singular batch. 

In [11]:
#plt.figure(figsize = (16,8))

#plt.subplot(1,2,1)
#plt.imshow(test_inputs[3])

#plt.subplot(1,2,2)
#plt.imshow(test_val[3])
#plt.show()


In [12]:
#siamese_model.save('siamese_model.keras')
#model = tf.keras.models.load_model('siamese_model.keras', custom_objects = {'L1Dist': L1Dist})

In [55]:
model.predict([test_inputs, test_val])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 796ms/step


array([[7.6874005e-07],
       [1.0000000e+00],
       [3.3164984e-08],
       [9.9990344e-01],
       [9.6981407e-12],
       [1.0000000e+00],
       [5.1810750e-10],
       [9.9999982e-01],
       [1.0000000e+00],
       [5.6202262e-11],
       [1.0000000e+00],
       [4.9784902e-04],
       [9.9992746e-01],
       [1.0000000e+00],
       [2.1414872e-09],
       [9.4013084e-03]], dtype=float32)

### Building the verification function 

This function handles the real-time verification process by comparing a captured input image against all of the stored verification images. It takes the current frame, the trained model, and two threshold parameters as inputs.

The function iterates through each verification image, preprocessing both the input and verification images to the required format. The model then predicts the similarity between each pair, returning a value between 0 and 1. 

A detection count is calculated by counting how many predictions exceed the detection threshold. The overall verification score is determined by dividing successful detections by the total number of verification images. Finally, the person is considered verified if this score exceeds the verification threshold, providing a robust verification system that requires multiple positive matches rather than relying on a single comparison.

In [72]:
def verify(frame, model, verification_threshold, detection_threshold):
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_image')):
        input_image = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        verification_image = preprocess(os.path.join('application_data', 'verification_image', image))

        result = model.predict(list(np.expand_dims([input_image, verification_image], axis = 1)), verbose = 0)
        results.append(result)

    detection = np.sum(np.array(results) > detection_threshold)
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_image')))

    verified = verification > verification_threshold

    return results, verified

In [120]:
#import shutil
#checkpoint_path = os.path.join('application_data', 'verification_image', '.ipynb_checkpoints')
#if os.path.exists(checkpoint_path):
    #shutil.rmtree(checkpoint_path)

In [123]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[450:900, 750:1200, :]

    cv2.imshow('Image verification', frame)
    
    key = cv2.waitKey(10) & 0xFF 
    if key == ord('v'): 
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        results, verified = verify(frame, model, 0.5, 0.7)
        print(verified)

    elif key == ord('q'):
        break

    
cap.release()
cv2.destroyAllWindows()

True
True
